In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
5,China Using Anal Swabs for COVID Testing WebMD,https://www.webmd.com/lung/news/20210128/china...,"Thu, 28 Jan 2021 14:52:00 GMT",2021-01-28 14:52:00+00:00
3,UNHCR's Approach to Evaluating the COVID-19 Re...,https://reliefweb.int/report/world/unhcr-s-app...,"Thu, 28 Jan 2021 13:37:00 GMT",2021-01-28 13:37:00+00:00
7,‘Immunity bubbles’? What you shouldn’t do afte...,https://www.wfla.com/community/health/coronavi...,"Thu, 28 Jan 2021 12:04:00 GMT",2021-01-28 12:04:00+00:00
4,How to get the COVID-19 vaccine from Walgreens...,https://www.wfla.com/community/health/coronavi...,"Thu, 28 Jan 2021 10:47:00 GMT",2021-01-28 10:47:00+00:00
1,COVID-19 Vaccine Tracker: How Many People Have...,https://www.npr.org/sections/health-shots/2021...,"Thu, 28 Jan 2021 10:31:01 GMT",2021-01-28 10:31:01+00:00
6,HAWAI'I COVID-19 JOINT INFORMATION CENTER DAIL...,https://governor.hawaii.gov/newsroom/hawaii-co...,"Thu, 28 Jan 2021 02:32:42 GMT",2021-01-28 02:32:42+00:00
0,South Carolina Officials Warn Public to Watch ...,https://scdhec.gov/news-releases/south-carolin...,"Wed, 27 Jan 2021 23:53:18 GMT",2021-01-27 23:53:18+00:00
2,The Covid-19 Vaccine-Development Multiverse - ...,https://www.nejm.org/doi/full/10.1056/NEJMc203...,"Wed, 27 Jan 2021 22:05:47 GMT",2021-01-27 22:05:47+00:00
8,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Wed, 27 Jan 2021 21:32:31 GMT",2021-01-27 21:32:31+00:00
9,KFF COVID-19 Vaccine Monitor | KFF Kaiser Fam...,https://www.kff.org/coronavirus-covid-19/dashb...,"Wed, 27 Jan 2021 14:09:03 GMT",2021-01-27 14:09:03+00:00
